In [14]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
from matplotlib.backends.backend_pdf import PdfPages
import os
import sys
import gdown

# Percorso alla tua cartella locale (senza Drive mount)
drive_path = './'

# Scarica i file necessari da GitHub nel percorso locale
!wget -O "Lettura_Dati.py" https://raw.githubusercontent.com/microlab-unibg/Sense2Gol-Pulse-vital-Parameters/Caricamento-file-e-collegamento-al-drive/MetodiDiRimozione/Lettura_Dati.py
!wget -O "background_ds.py" https://raw.githubusercontent.com/microlab-unibg/Sense2Gol-Pulse-vital-Parameters/Caricamento-file-e-collegamento-al-drive/MetodiDiRimozione/background_ds.py

# Assicurati che i file siano stati scaricati correttamente
assert os.path.exists('Lettura_Dati.py'), "Lettura_Dati.py non trovato!"
assert os.path.exists('background_ds.py'), "background_ds.py non trovato!"

# Importa i moduli dal percorso attuale
sys.path.insert(0, drive_path)
from Lettura_Dati import leggi_file
from background_ds import Background_ds

# Scarica il file specifico di dati da Google Drive usando gdown
file_url = "https://drive.google.com/uc?id=15LSYaKYgg9GrVJgwSE_dBTPDefqShAaD"
gdown.download(file_url, 'dati.txt', quiet=False)

# Leggi il file di input
file_path = "dati.txt"
frames = leggi_file(file_path)
frames_array = np.array(frames)

# Parametri
a = 10  # fattore di downsampling
Nm = 9  # numero di frame per blocco
T = 1 * Nm
N = (256 * Nm) // a
print(N)

# Ottieni il background come singolo array FFT da Background_ds
background = Background_ds(Nm, a)[:N//2]  # Prendi solo le frequenze positive della FFT

# Numero totale di finestre
total_windows = len(frames_array) // Nm

block_idx = 0
intervallo_tempo = 1

max_amplitudes_BRPM = []  # lista per memorizzare i respiri
max_amplitudes_BPM = []  # lista per memorizzare il battito

# Apri un PdfPages per salvare tutti i plot in un unico PDF
pdf_filename = "grafici_output.pdf"
with PdfPages(pdf_filename) as pdf:
    # Ciclo per eseguire il downsampling e il calcolo
    while intervallo_tempo <= total_windows:
        start_idx = block_idx
        end_idx = block_idx + Nm

        block_frames = frames_array[start_idx:end_idx]
        block_frames = block_frames.flatten()
        block_frames = block_frames[::a]
        combined_fft = np.fft.fft(block_frames, n=N)
        combined_fft_no_background = combined_fft[:N//2] - background
        xf = fftfreq(N, T/N)[:N//2]

        fft_amplitudes_selected = combined_fft_no_background[(xf > 0) & (xf <= 1)]
        max_amplitudes_BRPM_index = np.argmax(fft_amplitudes_selected)
        max_amplitudes_BRPM.append(np.abs(fft_amplitudes_selected[max_amplitudes_BRPM_index]))

        fft_amplitudes_selected2 = combined_fft_no_background[(xf > 0.66) & (xf <= 6.66)]
        max_amplitudes_BPM_index = np.argmax(fft_amplitudes_selected2)
        max_amplitudes_BPM.append(np.abs(fft_amplitudes_selected2[max_amplitudes_BPM_index]))

        # Crea il grafico e aggiungilo al PDF
        plt.figure(figsize=(8, 4))
        plt.plot(xf, np.abs(combined_fft_no_background))
        plt.title(f'Intervallo di tempo {intervallo_tempo} ({start_idx} - {end_idx})', fontsize=16)
        plt.xlabel('Frequenza [Hz]', fontsize=16)
        plt.ylabel('Ampiezza [au]', fontsize=16)
        plt.grid(True)

        # Salva la figura corrente nel PDF
        pdf.savefig()
        plt.close()

        block_idx += Nm
        intervallo_tempo += 1

# Calcola la media dei massimi valori di ampiezza
BRPM = np.mean(max_amplitudes_BRPM)
BPM = np.mean(max_amplitudes_BPM)
print("Media dei massimi valori di ampiezza tra 0 e 1 Hz:", BRPM)
print("BRPM:", (BRPM / 2) * 60)
print("Media dei massimi valori di ampiezza tra 0.66 e 6.66 Hz:", BPM)
print("BPM:", BPM * 60)

# Salva i risultati in un file di testo
output_filename = "risultati.txt"
with open(output_filename, "w") as f:
    f.write(f"Media dei massimi valori di ampiezza tra 0 e 1 Hz: {BRPM}\n")
    f.write(f"BRPM: {(BRPM / 2) * 60}\n")
    f.write(f"Media dei massimi valori di ampiezza tra 0.66 e 6.66 Hz: {BPM}\n")
    f.write(f"BPM: {BPM * 60}\n")

print("File dei risultati e PDF dei grafici generati con successo!")

import shutil
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Imposta il percorso del drive condiviso e delle sottocartelle
percorso_su_drive = '/content/drive/Shareddrives/Radar/Vital parameter monitoring/Sense2GoL_pulse_data/FileOutput/'

# Crea la directory se non esiste
os.makedirs(percorso_su_drive, exist_ok=True)

# Sposta i file su Google Drive
shutil.move("grafici_output.pdf", os.path.join(percorso_su_drive, "grafici_output.pdf"))
shutil.move("risultati.txt", os.path.join(percorso_su_drive, "risultati.txt"))

print("File caricati con successo su Google Drive nel drive condiviso Radar!")



--2024-11-07 19:58:52--  https://raw.githubusercontent.com/microlab-unibg/Sense2Gol-Pulse-vital-Parameters/Caricamento-file-e-collegamento-al-drive/MetodiDiRimozione/Lettura_Dati.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2371 (2.3K) [text/plain]
Saving to: ‘Lettura_Dati.py’

Lettura_Dati.py     100%[===================>]   2.32K  --.-KB/s    in 0s      

2024-11-07 19:58:52 (30.1 MB/s) - ‘Lettura_Dati.py’ saved [2371/2371]

--2024-11-07 19:58:52--  https://raw.githubusercontent.com/microlab-unibg/Sense2Gol-Pulse-vital-Parameters/Caricamento-file-e-collegamento-al-drive/MetodiDiRimozione/background_ds.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubus

Downloading...
From: https://drive.google.com/uc?id=15LSYaKYgg9GrVJgwSE_dBTPDefqShAaD
To: /content/dati.txt
100%|██████████| 771k/771k [00:00<00:00, 106MB/s]


230


Downloading...
From: https://drive.google.com/uc?id=1oF0mh5XYgx06jyDgADqXDVMGyvdEGKwO
To: /content/dati.txt
100%|██████████| 700k/700k [00:00<00:00, 83.6MB/s]


Media dei massimi valori di ampiezza tra 0 e 1 Hz: 0.47771539918471445
BRPM: 14.331461975541433
Media dei massimi valori di ampiezza tra 0.66 e 6.66 Hz: 0.5318030423754816
BPM: 31.908182542528895
File dei risultati e PDF dei grafici generati con successo!
Mounted at /content/drive
File caricati con successo su Google Drive nel drive condiviso Radar!


In [8]:
from google.colab import files
files.download("grafici_output.pdf")
files.download("risultati.txt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>